Prepare

In [ ]:
!pip install pygit2==1.12.2
%cd /content
!git clone https://github.com/mashb1t/Fooocus.git
%cd /content/Fooocus

Run

In [ ]:
!python entry_with_update.py --share --attention-split --disable-offload-from-vram --always-high-vram